In [ ]:
import torch
import torchvision
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Classification 

As the classification example we will use the "Hello World!" of machine learning: ["The MNIST Database of hanwritten digits"](http://yann.lecun.com/exdb/mnist/). 

## MNIST

This dataset bundled in many machine learning libraries and PyTorch is no exception. 

In [ ]:
train_data = torchvision.datasets.MNIST('./data/mnist', train=True, download=True)
test_data  = torchvision.datasets.MNIST('./data/mnist', train=False, download=True)

In [ ]:
train_features   = train_data.data.to(dtype=torch.float32)
train_labels = train_data.targets

The data consists of 28 by 28 pixels 8bit grayscale images of handwritten digits, the labels are integers denoting corresponding digits:

In [ ]:
fig_mnist, axes = plt.subplots(2,4, figsize=(16,8))
for i in range(8):
    ax=axes.ravel()[i]
    ax.imshow(train_features[i].numpy(), cmap='Greys');
    ax.set_title(train_labels[i].item(), fontsize=20)

For the purpose of this notebook I will load only a sybset of data. This will make the training of the network much quicker. 

In [ ]:
n_samples = 12000

Because we will be using the fully connected neural network as our classifier I will flatten each image to 28x28 long 1D array. I will also normalize the grayscale values to [0,1). 

In [ ]:
dataset = torch.utils.data.TensorDataset( 
    (train_features[:n_samples]/256.0).view(-1,28*28), 
    train_labels[:n_samples])

In [ ]:
train_dataset, validation_dataset = torch.utils.data.random_split(dataset, (10000,2000))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size = 100, 
                                           shuffle = True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, 
                                           batch_size = 100, 
                                           shuffle = True)

In [ ]:
test_features   = test_data.data.to(dtype=torch.float32)
test_labels = test_data.targets
test_dataset = torch.utils.data.TensorDataset(
    (test_features/256.0).view(-1,28*28), test_labels)

## Cross entropy

For classification problems  with $M$  possible categories $C=\{0,\ldots,M-1\}$ the output of the model  is a 1D vector with $M$ entries corresponding to probabilities  for each class

$$o^l_i = P(i | \b{x}_i)$$

where $l$ is the index of the  last layer. This is achieved by the _softmax_ activation function on the last layer:

$$o^{l}_i = \frac{ e^{\displaystyle o^{l-1}_i}}{\sum_{i=0}^{M-1}e^{\displaystyle o^{l-1}_i}}$$

Where $o^{l-1}_i$ is the output of the previous layer.  I will  use the word _layer_ in a generalized sense. A layer is a single operation so for example the  activation function application is considered as a separate layer. 

We will use the  _Negative Log Likelihood_ loss:

$$-\sum_{i=0}^N\log P(c_i|\b{x}_i) = -\sum_{i=0}^N\log a_{ c_{\scriptstyle i}}$$ 

where $c_i$  is the category corresponding to features $\b{x}_i$. 

This is often written in _cross entropy_ form:

$$-\sum_{i=0}^N
\sum_{j=0}^{M-1} l_{ij} \log a_{j}$$ 

where $l_{ij}$ are _one-hot_ encoded categories:

$$ l_{ij} =\begin{cases}
1 & c_i = j \\
0 & c_i\neq j
\end{cases}
$$

## The model 

We will use a fully  four  fully connected layers with `ReLU` activation layers in between as our model and `softmax` as the last layer.  The model can be easily constructed using the PyTorch `nn.Sequential` class:

In [ ]:
model = torch.nn.Sequential(
    nn.Linear(28*28,1200), nn.ReLU(),
    nn.Linear(1200,600), nn.ReLU(),
    nn.Linear(600,300), nn.ReLU(),
    nn.Linear(300,10), nn.Softmax(dim=1)
)

The network has 28x28=784 inputs and ten outputs which correspond to ten possible categories.  The model parameters: weights and biases are initalized randomly (more on this in other lectures). Let's check how this model performs, we will use   the `torch.no_grad()` _context manager_ to temporalily switch off gradient calculations

In [ ]:
with torch.no_grad():
    pred = model(train_dataset[:][0])

Tensor `pred` contains the predicted probabilities for each  digit for each input:

In [ ]:
pred[:4]

As we can see the distribution looks rather uniform. We can check that indeed the probabilities for each category sum to one:

In [ ]:
pred.sum(1)

The accuracy of  clasification can be calculated as follows: 

In [ ]:
def accuracy(pred, labels):
    return torch.sum(torch.argmax(pred,axis = 1)==labels).to(dtype=torch.float32).item()/len(labels)

Let's break down this function. The `argmax` function with `axis` argument equal to one for each row returns the index of the column containing the largest value. This is next compared with actual labels.   Finally we use implicit conversion of `False` to zero and `True` to one to calculate the number of labels predicted correctly. We finally divide by the length of the dataset to obtain accuracy. The conversion to float is needed because otherwise the integer arthmetic is used. 

Not suprisingly our accuracy is even worse then random guessing

In [ ]:
accuracy(pred, train_dataset[:][1])

We will  define another  accuracy function for further convenience: 

In [ ]:
def model_accuracy(model, dataset):
    features, labels = dataset[:]
    with torch.no_grad():
        pred = model(features)
    return accuracy(pred, labels)

Before we start training we need the loss function:

In [ ]:
def my_nll_loss(pred,labels):
    return -torch.mean(
        torch.log(0.0000001+pred[range(len(labels)),labels])
                      )

Let's break it down: the `pred[range(len(labels)),labels]`  expression takes from each row $i$ of the `pred` tensor the value from the column $\mathtt{labels}_i$ which is the probability of the correct label. We then take the logarithm and average over all examples. The small value is added in case one of the entries would be zero.  

After all this being said we can finally start training:

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
%%time
for e in range(5):
    for features, labels in train_loader:        
        optim.zero_grad()
        pred = model(features)
        loss = my_nll_loss(pred, labels)
        loss.backward()
        optim.step()   
    print(e, loss.item())        

Because logarthmic is a monotonicaly increasing function the accuracy functions will work even if the outputs do not represent probabilities. 

In [ ]:
model_accuracy(model, train_dataset)

As you can see the accuracy has increased greatly. But really important is the accuracy on the test data set:

In [ ]:
model_accuracy(model, test_dataset)

After training the model we can save it to file:

In [ ]:
torch.save(model,"mnist.pt")

and load later

In [ ]:
copy = torch.load("mnist.pt")

In [ ]:
with torch.no_grad():
    pred = torch.softmax(copy(train_dataset[:][0]),1)
    ac = torch.sum(torch.argmax(pred,1)==train_dataset[:][1]).to(dtype=torch.float32)/len(train_dataset)
ac 

## Using PyTorch loss functions

Our formulation of the loss function required calculation of the  logarithm of the softmax function. Doing those two operations separately is slower and numerically unstable. That's why PyTorch privides an implementation of the `log_softmax` function that does  both operations together. Please note that now the outputs of the model do not represent the probabilities.

In [ ]:
model = torch.nn.Sequential(
    nn.Linear(28*28,1200), nn.ReLU(),
    nn.Linear(1200,600), nn.ReLU(),
    nn.Linear(600,300), nn.ReLU(),
    nn.Linear(300,10), nn.LogSoftmax(dim=1)
)

We can now use the provided  negative likelihood loss function that expects the logarithms of probabilities as its input:

In [ ]:
nll_loss = torch.nn.NLLLoss()

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
%%time
for e in range(5):
    for features, labels in train_loader:        
        optim.zero_grad()
        pred = model(features)
        loss = nll_loss(pred,labels)
        loss.backward()
        optim.step()   
    print(e, loss.item())        

The accuracy functions will still work because the logarithm is monotonically increasing function.  

In [ ]:
model_accuracy(model, train_dataset)

In [ ]:
model_accuracy(model, test_dataset)

And finally we can drop the last activation layer

In [ ]:
model = torch.nn.Sequential(
    nn.Linear(28*28,1200), nn.ReLU(),
    nn.Linear(1200,600), nn.ReLU(),
    nn.Linear(600,300), nn.ReLU(),
    nn.Linear(300,10)
)

and use the cross entropy loss function that  calculates the log softmax internally

In [ ]:
ce_loss = torch.nn.CrossEntropyLoss()

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
%%time
for e in range(5):
    for features, labels in train_loader:        
        optim.zero_grad()
        pred = model(features)
        loss = ce_loss(pred,labels)
        loss.backward()
        optim.step()   
    print(e, loss.item())        

The accuracy functions will still work as before, because softmax  does not change the relative order of the input values.

In [ ]:
model_accuracy(model, train_dataset)

In [ ]:
model_accuracy(model, test_dataset)

## MSE loss

And finally we will use the MSE loss for comparison. To this end we have to one-hot encode the labels:

In [ ]:
one_hot_labels = torch.zeros(n_samples, 10).to(dtype=torch.float32)
one_hot_labels[range(n_samples),train_labels[:n_samples]] =  1.0

In [ ]:
one_hot_dataset = torch.utils.data.TensorDataset( 
    (train_features[:n_samples]/256.0).view(-1,28*28), 
    one_hot_labels)

In [ ]:
one_hot_train_dataset, one_hot_validation_dataset = torch.utils.data.random_split(one_hot_dataset,(10000,2000))

In [ ]:
one_hot_train_loader = torch.utils.data.DataLoader(one_hot_train_dataset, batch_size=100)

In [ ]:
model = torch.nn.Sequential(
    nn.Linear(28*28,1200), nn.ReLU(),
    nn.Linear(1200,600), nn.ReLU(),
    nn.Linear(600,300), nn.ReLU(),
    nn.Linear(300,10), nn.Softmax(dim=1)
)

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
mse_loss = torch.nn.MSELoss()

In [ ]:
%%time
for e in range(5):
    for features, labels in one_hot_train_loader:        
        optim.zero_grad()
        pred = model(features)
        loss = mse_loss(pred,labels)
        loss.backward()
        optim.step()   
    print(e, loss.item())        

In [ ]:
model_accuracy(model, train_dataset)

In [ ]:
model_accuracy(model, test_dataset)

As we can see the accuracy is much smaller: the convergence is slower. In the next notebook we will take a look at why it is so. 